In [1]:
import os

import cv2
import numpy as np
import requests
import torch
import torch.onnx
from torch import nn

### Define NN

In [2]:
class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor):
        super().__init__()
        self.upscale_factor = upscale_factor
        self.img_upsampler = nn.Upsample(
            scale_factor = self.upscale_factor,
            mode = 'bicubic',
            align_corners = False
        )
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.img_upsampler(x)
        out = self.relu(self.conv1(x))
        out = self.relu(self.conv2(out))
        out = self.conv3(out)
        return out

In [3]:
# download checkpoint and test image
urls = ['https://download.openmmlab.com/mmediting/restorers/srcnn/srcnn_x4k915_1x16_1000k_div2k_20200608-4186f232.pth', 
    'https://raw.githubusercontent.com/open-mmlab/mmediting/master/tests/data/face/000001.png'] 
names = ['srcnn.pth', 'face.png'] 
for url, name in zip(urls, names): 
    if not os.path.exists(name): 
        open(name, 'wb').write(requests.get(url).content) 

In [4]:
def init_torch_model():
    torch_model = SuperResolutionNet(upscale_factor=3)

    state_dict = torch.load('srcnn.pth')['state_dict']

    # Adapt the checkpoint
    for old_key in list(state_dict.keys()):
        new_key = '.'.join(old_key.split('.')[1:]) 
        state_dict[new_key] = state_dict.pop(old_key)
        
    torch_model.load_state_dict(state_dict)
    torch_model.eval()
    return torch_model
        
model = init_torch_model()
input_img = cv2.imread('face.png').astype(np.float32)

# HWC to NCHW
input_img = np.transpose(input_img, (2, 0, 1))
input_img = np.expand_dims(input_img, axis=0)

# inference
torch_output = model(torch.tensor(input_img)).detach().numpy()

# NCHW to HWC
torch_output = np.squeeze(torch_output, 0)
torch_output = np.clip(torch_output, 0, 255)
torch_output = np.transpose(torch_output, (1, 2, 0)).astype(np.uint8)

# show
cv2.imwrite('face_torch.png', torch_output)

/tmp/ipykernel_206863/2922022087.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('srcnn.pth')['state_dict']


True

In [5]:
x = torch.randn(1, 3, 256, 256)
with torch.no_grad():
    torch.onnx.export(
        model,
        x,#### 直接用Onnxruntime推理
        "srcnn.onnx",
        opset_version=11,
        input_names=["input"],
        output_names=["output"],
    )

#### 直接用Onnxruntime推理

In [ ]:
import onnxruntime as ort

# 获取推理器
ort_session = ort.InferenceSession("srcnn.onnx")
ort_inputs = {'input': input_img}

# 第一个参数是输出张量列表， 第二个为输入张量字典
# 对应导出时候的输入输出名称
ort_output = ort_session.run(['output'], ort_inputs)[0]

ort_output = np.squeeze(ort_output, 0)
ort_output = np.clip(ort_output, 0, 255)
ort_output = ort_output.transpose(1, 2, 0).astype(np.uint8)
cv2.imwrite('face_ort.png', ort_output)

True